In [ ]:
# # Install necessary packages
# !pip install --upgrade google-cloud-bigquery
# !pip install torch_geometric
# !pip install google-cloud-bigquery-storage==2.18.1
# !pip install --upgrade google-cloud-bigquery[pandas]

In [ ]:
import importlib
from google.colab import drive

drive.mount('/content/drive')

# Add the lib folder to the Python path
import sys
sys.path.append('/content/drive/My Drive/Expt_GNNs_MIMIC-iii/')

# Import the functions from PatientDataFetcher.py
from SBSCGM import PatientDataFetcher as main

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set parameters
project_id = 'forward-citron-385513'
dataset_name = 'physionet-data.mimiciii_demo'
# dataset_name = 'physionet-data.mimiciii_clinical'

fetcher = main.PatientDataFetcher(project_id, dataset_name)

In [ ]:
data_dict = fetcher.fetch_data()

Executing query: 
        SELECT DISTINCT subject_id FROM `physionet-data.mimiciii_demo.admissions`
        WHERE diagnosis LIKE '%SEPSIS%'
        
Fetched 8 unique subject IDs related to sepsis.
Executing query: SELECT * FROM `physionet-data.mimiciii_demo.patients` WHERE subject_id IN (10088, 41976, 40601, 10036, 10094, 10006, 10056, 10013)
Executing query: SELECT * FROM `physionet-data.mimiciii_demo.icustays` WHERE subject_id IN (10088, 41976, 40601, 10036, 10094, 10006, 10056, 10013)
Executing query: SELECT * FROM `physionet-data.mimiciii_demo.diagnoses_icd` WHERE subject_id IN (10088, 41976, 40601, 10036, 10094, 10006, 10056, 10013)
Executing query: SELECT * FROM `physionet-data.mimiciii_demo.procedures_icd` WHERE subject_id IN (10088, 41976, 40601, 10036, 10094, 10006, 10056, 10013)
Fetched 25 records from icustays.
Executing query: SELECT * FROM `physionet-data.mimiciii_demo.prescriptions` WHERE subject_id IN (10088, 41976, 40601, 10036, 10094, 10006, 10056, 10013)
Fetched 75 re

In [ ]:
# data_dict

In [ ]:
columns = data_dict.keys()
for each_key in columns:
  data_dict[each_key].drop(columns=['ROW_ID'], inplace=True)
  # data_dict[each_key].dropna(inplace=True)

  data_dict[each_key].reset_index(drop=True, inplace=True)
  data_dict[each_key].columns = [col.lower() for col in data_dict[each_key].columns]

  data_dict[each_key].to_csv(f'/content/drive/My Drive/Expt_GNNs_MIMIC-iii/data/mimiciii_demo_{each_key}.csv', index=False)
  print(each_key, data_dict[each_key].columns, data_dict[each_key].shape)

keys_to_delete = ['chart_events_info', 'input_events_info', 'output_events_info']

for key in keys_to_delete:
    if key in data_dict:
        del data_dict[key]

patient_info Index(['subject_id', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag'],
      dtype='object') (8, 7)
icu_info Index(['subject_id', 'hadm_id', 'icustay_id', 'dbsource', 'first_careunit',
       'last_careunit', 'first_wardid', 'last_wardid', 'intime', 'outtime',
       'los'],
      dtype='object') (25, 11)
diagnoses_info Index(['subject_id', 'hadm_id', 'seq_num', 'icd9_code'], dtype='object') (407, 4)
procedures_info Index(['subject_id', 'hadm_id', 'seq_num', 'icd9_code'], dtype='object') (75, 4)
prescriptions_info Index(['subject_id', 'hadm_id', 'icustay_id', 'startdate', 'enddate',
       'drug_type', 'drug', 'drug_name_poe', 'drug_name_generic',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'dose_val_rx',
       'dose_unit_rx', 'form_val_disp', 'form_unit_disp', 'route'],
      dtype='object') (1764, 18)
lab_events_info Index(['subject_id', 'hadm_id', 'itemid', 'charttime', 'value', 'valuenum',
       'valueuom', 'flag'],
      dtype='obj